In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import utils 
utils.init_logging()

import polars as pl
from polars import col, lit
import altair as alt
import simrs
from scenarios import builder as b
import scenarios
import cli
import analysis
from asgiref.sync import async_to_sync

In [7]:
from replay_cache import Replay
import replay_cache
import scenarios.examples as ex

# x = ex.small_replay()

# b.save_scenario(x, "basic")
await cli.run_channel("last")
# replay.agents
replay_cache.get("last").agents

behavior,cargo,coins,id,pos,tick
str,str,f64,str,str,i64
"""Exhaustive""",null,1000.0,"""a1""","""la""",0
"""Exhaustive""",null,999.9,"""a1""","""m1""",1
"""Exhaustive""",null,999.8,"""a1""","""m2""",2
"""Exhaustive""","""Wheat""",899.64995,"""a1""","""m3""",3
"""Exhaustive""",null,999.5,"""a1""","""m3""",4
"""Exhaustive""","""Wheat""",899.44995,"""a1""","""m2""",5
"""Exhaustive""",null,999.4,"""a1""","""m2""",6
"""Exhaustive""","""Wheat""",899.24995,"""a1""","""m3""",7
"""Exhaustive""",null,999.1,"""a1""","""m3""",8


In [ ]:
# scenario_input = scenarios.barbell_scenario(ticks=100, num_agents=10)
# (actions, agents, markets, events) = scenarios.run_scenario(scenario)

scenario = scenarios.load_scneario()
(actions, agents, markets, events) = (scenario.actions, scenario.agents, scenario.markets, scenario.events)

In [ ]:
print("agents") 
agents.head()

In [ ]:
print("markets") 
markets.head()

In [ ]:
print("actions")
actions.head(10)

In [ ]:
events.head(10)

In [ ]:
analysis.plot_agents(agents)

In [ ]:
analysis.plot_agent_locations(agents)

In [ ]:
analysis.plot_prices_by_port(markets)

In [ ]:
# analysis.plot_prices_by_port(analysis.no_agent_markets(scenario_input)) 

In [ ]:
routes = analysis.make_routes(events)
routes

In [ ]:
blag = routes.with_columns(pl.struct("src", "dst").alias("route"))
blag.groupby("route").agg(pl.count()).sort("count", descending=True)

In [ ]:
analysis.plot_trades(routes)

In [ ]:
analysis.plot_buy_and_sell_prices(routes) + analysis.plot_trades(routes)

In [ ]:
curr = scenario
res = curr.markets.filter(curr.markets["tick"] == 0).select("price", "port")

utils.keyed_by(res, "port", "price")

## Port level analysis
- How much does each port trade?
- Trade volume bucketted 
- Volume in dollars 
- Biggest trading partners (other ports)

In [ ]:
## Total goods traded per port
events.groupby("port").agg(pl.sum("amt"))
# def plot_

Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)